### Retrieving pathology image files in CDA, and their relationship to specimens

This notebook explores some considerations related to querying and making effective use of pathology image files via the Cancer Data Aggregator API.

Apparently the following studies include pathology in a DICOM format (thanks Keyvan).

* CPTAC-LUAD (case IDs: 11LU013 through 11LU035, and C3L-0001 through 00093)
* CPTAC-LSCC

It is not certain that either of the above also have radiology images, but that can easily be determined by a query<sup>1</sup>.

<sup>1</sup>The fault in that logic is that the query would merely say whether there are radiology images present. It would not address the question of whether there are radiology images on these cases that should be present.

### Some questions for investigation - and what is expected
* Pathology images would generally expected to be attached to the specific specimen from which material was prepared for imaging. 
    *  The histopathology would provide useful information about the specimens from which proteomics or genomic data were obtained. For example, does the pathology image confirm that the proteomic data is likely to have come from tumor cells?
    * Can the pathology images be associated with proteomic or genomic data?
* Whole slide pathology images are stored as numerous DICOM files.
    * Structure - These reflect both different tiles in a grid, and multiple layered grids representing different levels of magnification. That structure is mapped into the DICOM model (study-series-image) in a specific way.
   * Does the CDA representation of IDC pathology image files allow navigation of the levels of magnification and location of images within the grid?
    * A large number of files has to be managed. Does CDA make that convenient for a user?
* How does all of the above compare with the representation of pathology images in certain studies in the GDC. In these studies a whole slide image is represented as an SVS file


### 1. Are pathology images attached to the appropriate specimen?
On the way to that we have some more mundane questions

#### For the studies and cases above how many image files are there?

Define a function to list imaging files at the top (Subject) level of the CDA/CRDC-H model

In [1]:
from cdapython import Q
import json
from collections import Counter
import pandas as pd

def listCaseImages(case_id):
    q1 = Q('id = "{}"'.format(case_id))
    r = q1.run()
    #print(r)
    if r.count > 0:
        categories = Counter()
        imaging_types = Counter()
        for f in r[0]['File']:
            dc = f['data_category']
            categories[dc] += 1
            if f['data_category'] == "Imaging":
                imaging_types[f['data_type']+'/'+f['file_format']] += 1
        #print(categories)
        print ("\nCase:{} Image counts and types at top (Subject) level".format(case_id))
        tcounts = []
        for t,c in imaging_types.items():
            l = t.split("/")
            tcounts.append([l[0],l[1],c])            
        df = pd.DataFrame(tcounts, columns=['Data type','File format','File count'])
        df.set_index('Data type', inplace=True)
        display(df)

/Users/forei/imfenvs/cda2/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Call our function on a number of specimens from the CPTAC-LUAD project.

In [2]:
for case_id in ['11LU013','11LU016','11LU022','11LU035']:
    print("Case id:{}".format(case_id))
    matches = listCaseImages(case_id)
    print('_'*80)

Case id:11LU013
Getting results from database

Total execution time: 1205 ms

Case:11LU013 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,5


________________________________________________________________________________
Case id:11LU016
Getting results from database

Total execution time: 1172 ms

Case:11LU016 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,5


________________________________________________________________________________
Case id:11LU022
Getting results from database

Total execution time: 1342 ms

Case:11LU022 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,5


________________________________________________________________________________
Case id:11LU035
Getting results from database

Total execution time: 1063 ms

Case:11LU035 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,5


________________________________________________________________________________


Per the details about image type codes [here](https://wiki.cancerimagingarchive.net/display/Public/DICOM+Modality+Abbreviations)<sup>2</sup> each of the above cases has 5 Slide Microscopy DICOM images.



These cases do indeed have pathology images
There are 5 per case.
They are associated with the case. As we have seen elsewhere in CDA, file references are rolled up to the higher level from lower levels (subject, specimen). They may well also exist at lower levels.

<sup>2</sup> Is there an EVS list for that? Or a CCDH service?
For now the following dictionary will be useful for decoding

In [3]:
import json
with open('./data/image_type_codes.json') as f:
    image_type_codes = json.loads(f.read())
    
image_type_codes['SM']

'Slide Microscopy'

Are there any files at a lower level?

First are there any IDC ResearchSubjects?

#### Are there imaging files within ResearchSubject?

In [4]:
class sysCounter:
    """A class to count the system from which CDA objects originate"""
    def __init__(self):
        self.systemCounter = Counter()

    def add(self, obj):
        for i in obj['identifier']:
            self.systemCounter[i['system']] += 1
    
    
myCounter = sysCounter()
q1 = Q('id = "11LU022"')
r = q1.run()
for res in r[0]['ResearchSubject']:
    myCounter.add(res)
    for f in res['File']:
        myCounter.add(f)
    for sp in res['Specimen']:
        myCounter.add(sp)
        for sf in sp['File']:
            myCounter.add(sf)

for t,c in myCounter.systemCounter.items():
    print("system:{} object count:{}".format(t,c))

Getting results from database

Total execution time: 1103 ms
system:GDC object count:319
system:PDC object count:486


The only Research Subjects and Specimens are for GDC and PDC.

Issue: Pathology image files are not associated with the specific specimen they were derived from. Reported as [cda-python/issues/102](https://github.com/CancerDataAggregator/cda-python/issues/102)

Notes for the model:
It is possible that the pathology image(s) was(were) associated with a specimen from which both proteomic and genomic data were obtained. This would be good to know.

If these images were subject to quantitative image analysis (e.g. in QIN) which extracted particular measurements which genomic or proteomic data could they legitimately be associated with?

Onwards. 

### 2. A large number of imaging files have to be managed. Does CDA make that convenient for a user?
The next reasonable question it what are these five images?

What can CDA tell us about that?
Unfortunately the following will be verbose - but we want to see everyting CDA can tell us.

In [5]:
sm_files = []
for file in r[0]['File']:
    if file['data_type'] == 'SM':
        sm_files.append(file)
print(json.dumps(sm_files, indent=3))

[
   {
      "id": "e22612ad-f70a-4901-b7e9-b04f225b3d8d",
      "identifier": [
         {
            "system": "IDC",
            "value": "e22612ad-f70a-4901-b7e9-b04f225b3d8d"
         }
      ],
      "label": "e22612ad-f70a-4901-b7e9-b04f225b3d8d.dcm",
      "data_category": "Imaging",
      "data_type": "SM",
      "file_format": "DICOM",
      "associated_project": "cptac_luad",
      "drs_uri": "gs://idc-open/e22612ad-f70a-4901-b7e9-b04f225b3d8d.dcm",
      "byte_size": null,
      "checksum": null
   },
   {
      "id": "b8c662fc-ac1e-4e64-b05b-bf47432cc135",
      "identifier": [
         {
            "system": "IDC",
            "value": "b8c662fc-ac1e-4e64-b05b-bf47432cc135"
         }
      ],
      "label": "b8c662fc-ac1e-4e64-b05b-bf47432cc135.dcm",
      "data_category": "Imaging",
      "data_type": "SM",
      "file_format": "DICOM",
      "associated_project": "cptac_luad",
      "drs_uri": "gs://idc-open/b8c662fc-ac1e-4e64-b05b-bf47432cc135.dcm",
      "byte_size

There's nothing really very specific there to help us manage things. The only thing that varies across the five files is the id.

DRS may come to our assistance here. What can it tell us about these five files?

We can take the same approach as we did in another notebook.

In [6]:
from fasp.loc import crdcDRSClient
drsClient = crdcDRSClient()
drs_id = sm_files[0]['id']
print(drs_id)
drsClient.getObject(drs_id)

e22612ad-f70a-4901-b7e9-b04f225b3d8d
{"msg":"No bundle found","status_code":404}



404

DRS does not actually know about these ids. However, there is a direct gs:// URI, and its name suggests it is open
    
We can use the Google cloud to look at these.

It's outside the scope of what we can show in this notebook, but when those five files were downloaded and opened in a DICOM viewer it turned out that there were actually 3297 images present. CDA makes the management of the large number of files manageable by the user by not exposing that level of granularity. In actuality it likely inherits that encapsulation from IDC, but hold that thought.

### 3. Does the CDA representation of IDC pathology image files allow navigation of the levels of magnification and location of images within the grid?

No it doesn't, but that is a good level of isolation. That the large number of pathology tiles is hidden from the CDA user is useful. The navigation of the image pyramid is best left to a dedicated DICOM viewer. 

There still seem to be five files per slide though. The useful level of granularity for CDA pathology may well be the DICOM study. One study representing one slide image. This would correspond to the slide/svs granularity in GDC.

What can the IDC tell us about these files?

Here is a screenshot from IDC of a number of cases including C3L-00001. It has two DICOM studies.
![C3L-00001](images/C3L-00001.jpg)

What does CDA tell us about images from that case.

In [7]:
listCaseImages("C3L-00001")

Getting results from database

Total execution time: 1107 ms

Case:C3L-00001 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,10


We've seen a pattern where there are 5 images per case. Here we have 10 SM images and 2 studies. This looks like we have two microscope slide images - each study perhaps representing one slide image.
However, what do these images represent? How do they relate to the samples used for proteomics and genomics?

Getting to more advanced examples. Here is a screenshot from IDC which includes case C3L-00263.
![09CO022](images/C3L-00263.jpg)
What does CDA tell us about the images for this case?

In [8]:
listCaseImages("C3L-00263")

Getting results from database

Total execution time: 1119 ms

Case:C3L-00263 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,40
CR,DICOM,3


11 studies, guessing this is 
8 slide studies (5 images each)
3 CR studies

Case C3L-02834 is more complicated still, there a total of four imaging data types.

In [9]:
listCaseImages("C3L-02834")

Getting results from database

Total execution time: 938 ms

Case:C3L-02834 Image counts and types at top (Subject) level


,File format,File count
Data type,,
SM,DICOM,30
CR,DICOM,9
NM,DICOM,27
DX,DICOM,2


As a CDA user one is not necessarily an imaging expert which is where our custom decoder for image type codes comes in handy:

In [10]:
for code in ['SM','CR','NM','DX']:
    print("{} {}".format(code,image_type_codes[code]))

SM Slide Microscopy
CR Computed Radiography
NM Nuclear Medicine
DX Digital Radiography


The above begs a number of questions from the CDA user's point of view.

* When were these imaging procedures performed in relation to other data types from the same subjects?
* If we have 6 or 8 microscopy slides how do we relate them to the specimens that CDA knows about?

These questions cannot be answered because all the images are at the top level.

How hard is it to do record things at the right level? How much does it need to complicate the model? Probably not very much, and the IDC (DICOM) model already has the details. That is probably worth some investigation, perhaps in another notebook.

For now the issue is logged as [cda-python/issues/103](https://github.com/CancerDataAggregator/cda-python/issues/103)

### 4. How CDA/IDC representation compare with the representation of pathology images in certain studies in the CDA/GDC.

We know that there are pathology images stored in the GDC for TCGA cases. These can be used as follows to show
* an alternate approach to relating pathology to the specific samples from which they came
* that a similar data model is desirable whichever node (GDC, IDC) the pathology images are in

Taking case TCGA-28-1750 as an example.

First, as a cross check are there any IDC files for this case (at the top level).

In [11]:
gbm_case = "TCGA-28-1750"
listCaseImages(gbm_case)

Getting results from database

Total execution time: 1124 ms

Case:TCGA-28-1750 Image counts and types at top (Subject) level


,File format,File count
Data type,,


No there are no IDC files for this case.

Next we will look for the pathology images for this case within the what CDA returns.

In [12]:
typeCounter = Counter()
q1 = Q('id = "{}"'.format(gbm_case))
r = q1.run()
for subf in r[0]['File']:
    typeCounter[subf['file_format']] += 1
for res in r[0]['ResearchSubject']:
    for f in res['File']:
        typeCounter[f['file_format']] += 1
    for sp in res['Specimen']:
        for sf in sp['File']:
            typeCounter[sf['file_format']] += 1
typeCounter


Getting results from database

Total execution time: 1041 ms


Counter({'MAF': 96,
         'BCR Biotab': 34,
         'TXT': 68,
         'VCF': 96,
         'BCR XML': 4,
         'BCR SSF XML': 2,
         'BAM': 14,
         'TSV': 4})

No Pathology images/SVS files there. It seems these files from GDC are not available through the CDA. Reported as [cda-python/issues/101](https://github.com/CancerDataAggregator/cda-python/issues/101)

Nevertheless, the CDA does have the specimens with which those images should be properly associated. The following lists them.

In [13]:
for res in r[0]['ResearchSubject']:
    for sp in res['Specimen']:
        if sp['specimen_type'] == 'slide':
            print(sp['id'], sp['source_material_type'])

106a0ec8-0e9d-40e9-a8b0-71b9c92c940c Primary Tumor
61384423-4581-4ade-a904-a57e73e2654f Primary Tumor
fb126dbb-0802-4150-9a66-fbe6265efd18 Primary Tumor


### Dumping the whole output for a few cases
For convenience the following produces full dumps to file of some of the cases explored in this workbook. The actual output has not been loaded to GitHub. Please run the notebook to produce your own copies.

In [14]:
def dumpSubject(subject_id):
    q1 = Q('id = "{}"'.format(subject_id))
    r = q1.run()
    for res in r:
        with open('./query_results/{}.json'.format(subject_id), 'w') as outfile:
            json.dump(res, outfile)

In [15]:
dumpSubject('11LU022')

Getting results from database

Total execution time: 1301 ms


In [16]:
dumpSubject('C3L-0001')       

Getting results from database

Total execution time: 679 ms


In [17]:
dumpSubject(gbm_case)

Getting results from database

Total execution time: 1013 ms
